In [1]:
import math
import numpy as np
import pickle
%matplotlib inline
import matplotlib.pyplot as plt
import cv2
from scipy.misc import imresize
from skimage import feature

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint as sp_randint

In [2]:
# helpers

# plot images, list are dislayed in subplots
def plot(imgs, title=None):
    # make sure input is a list
    if not isinstance(imgs, list):
        imgs = [imgs]
    
    plt.figure()
        
    for i in range(len(imgs)):
        if(title is not None): 
            plt.suptitle(title)
        plt.subplot(1, len(imgs), i+1)
        plt.axis("off")
        plt.imshow(imgs[i])
        

# create empty list of given size
def empty(size):
    return [None]*size

print('done')

done


In [12]:
# choose data_set to be used
data_set = 'train'

# load data
data = pickle.load(open("./data_pp/data-%s.pkl" % data_set, 'rb'))
c = len(data['rgb'])
print('%i samples' % c)

r = range(c)

print('done')

38000 samples
done


In [4]:
features = {'hog_dep':empty(c),'hog_depd':empty(c),'hog_rgb':empty(c)}

In [5]:
for i in r:
    if(not i%1000):
        print(i)
    
    if(not data['valid'][i]):
        continue 
        
    dep = data['dep'][i]
    dep[dep==0] = 255
    
    d = 127
    while((dep<d).sum() > 500):
        d -= 1
    
    orientations=8
    pixels_per_cell=(20,20)
    cells_per_block=(2,2)
    
#     features['hog_dep'], hog_dep = feature.hog(dep, orientations, pixels_per_cell, cells_per_block, visualise=True)
    features['hog_dep'][i] = feature.hog(dep, orientations, pixels_per_cell, cells_per_block, visualise=False)
#     features['hog_depd'], hog_depd = feature.hog(dep<d, orientations, pixels_per_cell, cells_per_block, visualise=True)
    features['hog_depd'][i] = feature.hog(dep<d, orientations, pixels_per_cell, cells_per_block, visualise=False)
    
    rgb = data['rgb'][i]
    rgb = cv2.cvtColor(rgb, cv2.COLOR_RGB2GRAY)
    
#     features['hog_rgb'], hog_rgb = feature.hog(rgb, orientations, pixels_per_cell, cells_per_block, visualise=True)
    features['hog_rgb'][i] = feature.hog(rgb, orientations, pixels_per_cell, cells_per_block, visualise=False)
    
#     plot([hog_dep, hog_depd, hog_rgb])

print('done')

0
1000
2000
3000
4000
5000
6000
7000
8000
done


In [7]:
X = np.zeros((c, 2400))

for i in r:
    X[i, 0:800] = features['hog_dep'][i]
    X[i, 800:1600] = features['hog_depd'][i]
    X[i, 1600:2400] = features['hog_rgb'][i]
    
print(X.shape)

(8190, 2400)


In [9]:
X[np.isnan(X)] = 0

In [15]:
Y = data['label']
print(Y.shape)

(38000,)


In [10]:
pickle.dump(X, open("./features/features-%s.pkl" % data_set, 'wb'))
pickle.dump(Y, open("./features/labels-train.pkl", 'wb'))